In [ ]:
import json
import os
import shutil
import uuid
from datetime import datetime
from pathlib import Path
from pprint import pprint

from tqdm.auto import tqdm

from .. import evaluate, inference

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["nnUNet_def_n_proc"] = "4"
# os.environ["OMP_NUM_THREADS"] = "1"
MODEL_OUT_PATH = Path("./tmp/predictions")
EVAL_OUT_PATH = Path("./tmp/output")

SAM2_CHECKPOINT = Path(
    "/rodata/mnradonc_dev/m299164/trackrad/model-training/sam2/sam2_logs/configs/sam2.1_training/sam2.1_hiera_s_trackrad_08_04_mri_ext_copy.yaml/checkpoints/checkpoint.pt"
)

INPUT_PATH = Path(
    "/rodata/mnradonc_dev/m299164/trackrad/datasets/trackrad2025/trackrad2025_labeled_training_data"
)


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [2]:
shutil.rmtree("./tmp", ignore_errors=True)


def prediction_json(
    cohort: str, job_id: str, case_id: str, start_time: str, end_time: str
) -> str:
    return json.dumps(
        {
            "pk": job_id,
            "ground_truth_directory": str(INPUT_PATH),
            "input_directory": str(MODEL_OUT_PATH / cohort),
            "output_directory": str(EVAL_OUT_PATH / cohort),
            "inputs": [
                {"value": 8, "interface": {"slug": "frame-rate"}},
                {"value": 1.5, "interface": {"slug": "magnetic-field-strength"}},
                {"value": "abdomen", "interface": {"slug": "scanned-region"}},
                {
                    "image": {"name": "mri-linac-target.mha"},
                    "interface": {
                        "slug": "mri-linac-target",
                        "relative_path": "images/mri-linac-target",
                    },
                },
                {
                    "image": {"name": f"{case_id}.mha"},
                    "interface": {
                        "slug": "mri-linac-series",
                        "relative_path": "images/mri-linacs",
                    },
                },
            ],
            "status": "Succeeded",
            "outputs": [
                {
                    "image": {"name": "output.mha"},
                    "interface": {
                        "slug": "mri-linac-series-targets",
                        "relative_path": "images/mri-linac-series-targets",
                    },
                }
            ],
            "started_at": start_time,
            "completed_at": end_time,
        }
    )

In [3]:
NN_UNET_MODEL_FOLDER = Path(
    "/rodata/mnradonc_dev/m299164/trackrad/datasets/nnUNet/nnUNet_results/Dataset471_TrackRadSmooth5/nnUNetTrainerDAMike__nnUNetPlans__2d"
)
NNUNET_FOLDS = (0,)

if MODEL_OUT_PATH.exists():
    shutil.rmtree(MODEL_OUT_PATH)
if EVAL_OUT_PATH.exists():
    shutil.rmtree(EVAL_OUT_PATH)

os.environ["TQDM_DISABLE"] = "1"


def do_inference(cohort: str, do_refinement: bool):
    for case_folder in tqdm(list(INPUT_PATH.iterdir())):
        if not case_folder.is_dir():
            continue

        print(f"Processing case: {case_folder}")

        job_id = str(uuid.uuid4())
        output_path = MODEL_OUT_PATH / cohort / job_id / "output"

        start_time = datetime.now().isoformat()
        inference.run(
            case_folder,
            output_path,
            sam2_checkpoint=SAM2_CHECKPOINT,
            nnunet_model_folder=NN_UNET_MODEL_FOLDER,
            nnunet_folds=NNUNET_FOLDS,
            do_refinement=do_refinement,
        )
        end_time = datetime.now().isoformat()

        json_file = output_path / "prediction.json"
        json_file.parent.mkdir(parents=True, exist_ok=True)
        json_file.touch(exist_ok=True)
        with open(json_file, "w") as f:
            f.write(
                prediction_json(
                    cohort,
                    job_id,
                    case_folder.name,
                    start_time,
                    end_time,
                )
            )


do_inference("sam2-only", do_refinement=False)


  0%|          | 0/50 [00:00<?, ?it/s]

Processing case: /rodata/mnradonc_dev/m299164/trackrad/datasets/trackrad2025/trackrad2025_labeled_training_data/A_026
Runtime loading:   4.79176 s


frame loading (JPEG): 100%|██████████| 100/100 [00:02<00:00, 47.15it/s]
/rodata/mnradonc_dev/m299164/trackrad/trackrad-model/.venv/lib/python3.11/site-packages/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/rodata/mnradonc_dev/m299164/trackrad/trackrad-model/.venv/lib/python3.11/site-packages/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 100/100 [00:02<00:00, 48.49it/s]


Runtime algorithm: 5.40249 s
Runtime writing:   0.01080 s
Processing case: /rodata/mnradonc_dev/m299164/trackrad/datasets/trackrad2025/trackrad2025_labeled_training_data/A_010
Runtime loading:   1.69308 s


propagate in video: 100%|██████████| 239/239 [00:05<00:00, 47.47it/s]


...

Runtime algorithm: 2.75598 s
Runtime writing:   0.00933 s
Processing case: /rodata/mnradonc_dev/m299164/trackrad/datasets/trackrad2025/trackrad2025_labeled_training_data/B_010
Runtime loading:   1.45998 s


propagate in video: 100%|██████████| 93/93 [00:01<00:00, 46.52it/s]


Runtime algorithm: 5.61432 s
Runtime writing:   0.01243 s
Processing case: /rodata/mnradonc_dev/m299164/trackrad/datasets/trackrad2025/trackrad2025_labeled_training_data/B_006
Runtime loading:   1.42325 s


propagate in video: 100%|██████████| 97/97 [00:02<00:00, 46.97it/s]


Runtime algorithm: 5.84966 s
Runtime writing:   0.01409 s


In [5]:
for model_out_path in [MODEL_OUT_PATH / "sam2-only"]:
    predictions = []
    for json_file in model_out_path.rglob("*.json"):
        with json_file.open("r") as infile:
            content = json.loads(infile.read().strip())
            if content:
                predictions.append(content)

    output_file = model_out_path / "predictions.json"
    with output_file.open("w") as outfile:
        json.dump(predictions, outfile)

    eval_out_path = EVAL_OUT_PATH / model_out_path.name
    eval_out_path.mkdir(parents=True, exist_ok=True)
    evaluate.main(input_directory=model_out_path, output_directory=eval_out_path)

Input Files:
['tmp/predictions/baseline/predictions.json',
 'tmp/predictions/baseline/3e885678-51a8-4677-8a9c-c5e5ec624439/output/prediction.json',
 'tmp/predictions/baseline/3e885678-51a8-4677-8a9c-c5e5ec624439/output/images/mri-linac-series-targets/output.mha',
 'tmp/predictions/baseline/69c5342d-3d94-4ede-b0ee-04687f59eb55/output/prediction.json',
 'tmp/predictions/baseline/69c5342d-3d94-4ede-b0ee-04687f59eb55/output/images/mri-linac-series-targets/output.mha',
 'tmp/predictions/baseline/264fe778-efc2-4ca5-b1e7-6c692b630623/output/prediction.json',
 'tmp/predictions/baseline/264fe778-efc2-4ca5-b1e7-6c692b630623/output/images/mri-linac-series-targets/output.mha',
 'tmp/predictions/baseline/272120b3-779e-4739-aabc-6fba33a47a11/output/prediction.json',
 'tmp/predictions/baseline/272120b3-779e-4739-aabc-6fba33a47a11/output/images/mri-linac-series-targets/output.mha',
 'tmp/predictions/baseline/40fbbd42-6b6d-4967-ae2e-f72f93119edb/output/prediction.json',
 'tmp/predictions/baseline/40fbb

In [ ]:
baseline_metrics_file = EVAL_OUT_PATH / "sam2-only" / "metrics.json"
with baseline_metrics_file.open("r") as f:
    baseline_metrics = json.load(f)
print("sam2-only:")
pprint(baseline_metrics["aggregates"])


baseline_dice = {
    r["case_id"]: r["dice_similarity_coefficient"] for r in baseline_metrics["results"]
}

baseline:
{'center_distance': 3.733786919889287,
 'dice_similarity_coefficient': 0.8801941977957676,
 'hausdorff_distance_95': 5.715335896202309,
 'loading_time': 1.898943533735071,
 'relative_d98_dose': 0.8903327089611278,
 'runtime': 5.720219446264929,
 'surface_distance_average': 2.414704306000231,
 'time_per_frame': 0.056894961669633246,
 'total_time': 380.9581490000001}


In [ ]:
# after = {
#     "center_distance": 1.3046445819475643,
#     "dice_similarity_coefficient": 0.920104286178238,
#     "hausdorff_distance_95": 3.1441829912556383,
#     "loading_time": 1.8490781539150303,
#     "relative_d98_dose": 0.9677078721474955,
#     "runtime": 5.5294807060849696,
#     "surface_distance_average": 1.1402019564120371,
#     "time_per_frame": 0.054997818839118445,
#     "total_time": 368.927943,
# }

# before = {
#     "center_distance": 1.3313475618149924,
#     "dice_similarity_coefficient": 0.9186995586692427,
#     "hausdorff_distance_95": 3.2305672318119276,
#     "loading_time": 1.7498454130518974,
#     "relative_d98_dose": 0.9613066590962253,
#     "runtime": 5.658172566948103,
#     "surface_distance_average": 1.1612007734907186,
#     "time_per_frame": 0.05627782541225483,
#     "total_time": 370.4008989999999,
# }

# %matplotlib inline
# import matplotlib.pyplot as plt

# metrics_to_plot = [
#     "dice_similarity_coefficient",
#     "center_distance",
#     "hausdorff_distance_95",
#     "surface_distance_average",
#     "relative_d98_dose",
# ]

# changes = [(before[m] - after[m]) / before[m] for m in metrics_to_plot]
# changes[0] = -changes[0]
# changes[4] = -changes[4]

# plt.figure(figsize=(8, 4))
# bars = plt.bar(metrics_to_plot, changes)
# plt.ylabel("Change (After - Before)")
# plt.title("Metric Change After vs Before")
# plt.axhline(0, color="gray", linestyle="--", linewidth=1)
# plt.xticks(rotation=30, ha="right")
# plt.tight_layout()
# plt.show()